In [6]:
from gurobipy import *
import string
import time

#####  Benders  #####

C = {}
C[0] = 1.01
C[1] = 1.02
C[2] = 1.03
C[3] = 1.04
C[4] = 1.05
C[5] = 1.06
C[6] = 1.07
C[7] = 1.08
C[8] = 1.09
C[9] = 1.10

f = 1.045

b = {}
b[0] = 1000
for i in range(1,11):
    b[i] = 100
    
B = {}
B[0] = 1
for i in range(1,11):
    B[i] = 0

A = []
a1 = []
for i in range(10):
    a1.append(1)
A.append(a1)
for i in range(10):
    ai = []
    for j in range(10):
        if(i == j):
            ai.append(1)
        else:
            ai.append(0)  
    A.append(ai)
    
print(A)

#############################################

MP = Model('Master')
MP.modelSense = GRB.MAXIMIZE

z = MP.addVar(vtype=GRB.CONTINUOUS, obj = 1, name = 'z_{}')
y = MP.addVar(vtype=GRB.INTEGER, obj = 0, lb=0, name = 'y')


y1 = 150
f = 1.045

#SP.optimize()
#print(x)
#print(SP.display())
LB = -999999
UB = 999999
eps = 0.01

numberofx = 10
SP = Model('Subproblem')
SP.modelSense = GRB.MINIMIZE

u = {}
### numbrer of dual variables in subproblem = rows in A
for i in range(len(A)):
    u[i] = SP.addVar(vtype=GRB.CONTINUOUS, lb=0, name = 'u_{}'.format(i))

sc = {}
#### number of sonstraints of subproblem = number of elements in C
for i in range(len(C)):
    sc[i] = SP.addConstr(quicksum(A[j][i]*u[j] for j in range(len(u)))>= C[i])


#####
itera = 1

SP.setParam(GRB.Param.InfUnbdInfo,1) #not to stop even if unbounded
MP.setParam(GRB.Param.InfUnbdInfo,1)
ActualObj = 0
while(UB-LB >= eps):
    print(itera)
    itera+=1
    SP._u = u
    SP.setObjective(quicksum((b[j]-B[j]*y1)*u[j] for j in range(len(u))), GRB.MINIMIZE)
    SP.optimize() 
    if(SP.status == GRB.OPTIMAL):        
        objec = 0
        for j in range(len(u)):
            objec += (b[j]-B[j]*y1)*u[j].x
        objec += f*y1     
        LB = max(LB, objec)
        print(LB)
        MP.addConstr(quicksum((b[j]-B[j]*y)*u[j].x for j in range(len(b)))+ f*y>=z)
        print("\n Optimality Cut added")
    elif(SP.status == GRB.UNBOUNDED):
        ray = SP.UnbdRay #obtain extreme ray
        MP.addConstr(quicksum((b[j]-B[j]*y)*ray[j] for j in range(len(b)))>= 0)
        print("\n Feasibility Cut added")
    else:
        print(SP.status)
        break
    print(MP.display())
    MP.optimize()
    UB = MP.objVal
    y1 = y.x
    print(y1)

print("####################################################################")
print("The optimal objective:")
print(UB)
print("The optimal y:")
print(y)
print("The optimal x:")
#dual of the subproblem constraints
for j in range(len(sc)):
    print(sc[j].pi)

[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]]
Set parameter InfUnbdInfo to value 1
Set parameter InfUnbdInfo to value 1
1
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 10 rows, 11 columns and 20 nonzeros
Model fingerprint: 0x8120088f
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+02, 9e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.00s
Presolved: 10 rows, 11 columns, 20 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.055000e+01   0.000000e+00      0

Thread count was 8 (of 8 available processors)

Solution count 3: 1064.75 1061.76 1061.75 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.064750000000e+03, best bound 1.064750000000e+03, gap 0.0000%
350.0
5
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 10 rows, 11 columns and 20 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+02, 7e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -1.5000000e+32   6.000000e+30   1.500000e+02      0s
       3    6.9700000e+02   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.00 seconds (0.00 work units)
Optimal objective  6.970000000e+02
1062.75

 Optimality Cut added
Maximize
  <gurobi.LinExpr: z_{}>
Subject To
  R0: <gurobi.LinExpr: -1.0 z_{} + 0.02499999999999991 y> >= -1056
